## SRJ

## NH3 - Ham verification

    Update 

    1. In this, I converted the A-code to select the active space, it selected 2 orbitals [4,5]. I am unable to match 
        the data. Core and active decides the core constant value  
          
    2. when using sto-3g basis, and core = [0,1,2,3,6,7] core_constant = -49.6333072, 
        Final energy after A = -47.30589893057234, A-code : -54.8225228954901

    3. core = [0,1,2,3] , core_constant =  -53.27662334, Final A energy = -54.78634331314973, Acode: -54.8225228954901

    4. From mf.mo_occ data, we find the [6,7] have 0 occupancy, hence cannot be treated as core. 

    5. In A-code, when using mf.mo_coeffs, we find our frozen core energy matches and final energy matches with A-code. 
       Our first goal = Frozen energy matched. Adapt code = matched

    6. In ASF, they are changing the mf.mo_coeff to active_space.mo_coeff, and getting the result

    7. I verified this case for all basis set : sto-3g, sto-6g and ANO-RCC-MB for NH3 as well

In [ ]:
'N 0 0 0; H 0 0 1.40; H 0 0.60 -0.336000; H 0.60 0.0 -0.336000'

In [4]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    core, active = qml.qchem.active_space(electrons, orbitals, active_electrons=active_electrons, active_orbitals=active_orbitals) # Spatial orbitals
    print('core orbitals:', core)
    print('active orbitals:', active)
    #print(H)
    return core, active

In [5]:
symbols  = [ 'N', 'H', 'H', 'H']
print('H2O-0.6A-GS+ES-DBS')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 1.40*r_bohr], [0, 0.60*r_bohr, -0.336*r_bohr], [0.60, 0, -0.336*r_bohr]])

print(coordinates)

electrons = 10  # 7 from N and 3 from H
orbitals = 16 #Spin orbitals  
charge = 0

active_electrons = 2 #No of electrons
active_orbitals = 2 #Spatial orbitals

core, active = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=1)

H2O-0.6A-GS+ES-DBS
[[ 0.          0.          0.        ]
 [ 0.          0.          2.64561638]
 [ 0.          1.13383559 -0.63494793]
 [ 0.6         0.         -0.63494793]]
Using active space, check if you change the H accordingly
core orbitals: [0, 1, 2, 3]
active orbitals: [4, 5]


    Using ANO-RCC-MB for NH3

In [1]:
core = [0,1,2,3]
active = [4,5]

active_orbitals = 2 #Spatial 
active_electrons = 2 

In [2]:
import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['N', 'H'], fmt='nwchem')

import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

# Your original setup (unchanged) 
mol_pyscf = gto.M(
    atom = 'N 0 0 0; H 0 0 1.40; H 0 0.60 -0.336000; H 0.60 0.0 -0.336000',
    basis = basis_dict,
    spin = 0)
rhf = scf.RHF(mol_pyscf)
rhf.kernel()

import pennylane as qml 
from pennylane import qchem


one_ao = mol_pyscf.intor_symmetric("int1e_kin") + mol_pyscf.intor_symmetric("int1e_nuc")
two_ao = mol_pyscf.intor("int2e_sph")


one_mo = np.einsum("pi,pq,qj->ij", rhf.mo_coeff, one_ao, rhf.mo_coeff, optimize=True)
two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

core_constant = np.array([rhf.energy_nuc()])
print('The value of core:', core_constant)

for i, (e, occ) in enumerate(zip(rhf.mo_energy, rhf.mo_occ)):
    print(f"MO #{i:2d}: Energy = {e:10.6f} Ha, Occupancy = {occ}")

# convert the two-electron integral tensor to the physicists’ notation
two_mo = np.swapaxes(two_mo, 1, 3)

#Changed core and active orbitals


total_orb = len(core) + len(active)
print('Total orbitals are',total_orb)

# define the active space and recompute the integrals
def pyscf_integral(core, active, one_mo, two_mo, core_constant):
    if core and active:
        for i in core:
            print('The value of i', i)
            core_constant = core_constant + 2 * one_mo[i][i]
            for j in core:
                #core_constant = core_constant + 2 * two_mo[i][j][j][i] - two_mo[i][j][i][j]
                core_constant = core_constant + 2 * two_mo[i][j][j][i] - two_mo[i][j][i][j]
                print('core constant is', core_constant)
        for p in active:
            for q in active:
                for i in core:
                    #print(f'The value of p and q', {p}, {q})
                    one_mo[p, q] = one_mo[p, q] + (2 * two_mo[i][p][q][i] - two_mo[i][p][i][q])
    
        one_mo = one_mo[qml.math.ix_(active, active)]
        two_mo = two_mo[qml.math.ix_(active, active, active, active)]

    return core_constant, one_mo, two_mo

core_constant, one_mo, two_mo = pyscf_integral(core, active, one_mo,two_mo, core_constant)

H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)

H = qml.jordan_wigner(H_fermionic)
print(H)


converged SCF energy = -55.4273773558637
The value of core: [14.61897422]
MO # 0: Energy = -15.553630 Ha, Occupancy = 2.0
MO # 1: Energy =  -1.349952 Ha, Occupancy = 2.0
MO # 2: Energy =  -0.688361 Ha, Occupancy = 2.0
MO # 3: Energy =  -0.615870 Ha, Occupancy = 2.0
MO # 4: Energy =  -0.471265 Ha, Occupancy = 2.0
MO # 5: Energy =   0.229361 Ha, Occupancy = 0.0
MO # 6: Energy =   0.957696 Ha, Occupancy = 0.0
MO # 7: Energy =   1.069650 Ha, Occupancy = 0.0
Total orbitals are 6
The value of i 0
core constant is [-33.94526015]
core constant is [-32.18774961]
core constant is [-30.58263518]
core constant is [-29.14873825]
The value of i 1
core constant is [-41.4310061]
core constant is [-40.70933742]
core constant is [-39.57509224]
core constant is [-38.64774047]
The value of i 2
core constant is [-49.27891749]
core constant is [-48.14467231]
core constant is [-47.51797756]
core constant is [-46.57735128]
The value of i 3
core constant is [-56.21952182]
core constant is [-55.29217004]
core c

In [3]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
#warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(active_electrons, H, adapt_it):
    print('Using active space, check if you change the H accordingly')
    qubits = 2*active_orbitals
    active_electrons = active_electrons
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #hf_state = np.array([1,1,1,0,0,0,1,1,1,0,0,0])
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    #def callback(params):
        #print(f"Current parameters: {params}")
        #print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 


    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization

        #Energy calculation
        result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)



        ostate = new_state(hf_state, ash_excitation, params)
        #print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        print('\n') 
    return gs_state, params, ash_excitation, qubits, H, energies, old_grad






#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0




gs_state, params, ash_excitation, qubits, H,energies, old_grad = ags_exact( active_electrons, H, adapt_it=5) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


Using active space, check if you change the H accordingly
HF state is -55.42737735586377
Total excitations are 3
The adapt iteration now is 0
The highest operator value is 0.07869106190202811 for operator a⁺(0) a⁺(1) a(2) a(3)
Highest gradient excitation is [0, 1, 2, 3]
Final updated parameters: [0.08410854]
Final cost: -55.42903319816899


The adapt iteration now is 1
The highest operator value is 0.004949661322760376 for operator a⁺(0) a(2)
Highest gradient excitation is [0, 2]


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


Final updated parameters: [0.08530038 0.01821775]
Final cost: -55.42905574988393


The adapt iteration now is 2
The highest operator value is 0.003883036832467335 for operator a⁺(1) a(3)
Highest gradient excitation is [1, 3]


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


Final updated parameters: [0.0861087  0.0149709  0.01494908]
Final cost: -55.42907027670012


The adapt iteration now is 3
The highest operator value is 4.1696614552690775e-06 for operator a⁺(1) a(3)
Highest gradient excitation is [1, 3]


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


Final updated parameters: [8.61132800e-02 1.49684901e-02 1.49571027e-02 6.47614141e-06]
Final cost: -55.429070276720495


The adapt iteration now is 4
The highest operator value is 6.642604723841594e-07 for operator a⁺(0) a⁺(1) a(2) a(3)
Highest gradient excitation is [0, 1, 2, 3]
Final updated parameters: [ 8.61132800e-02  1.49684901e-02  1.49571027e-02  6.47614141e-06
 -1.41261037e-15]
Final cost: -55.4290702767202


The params after GS is [ 8.61132800e-02  1.49684901e-02  1.49571027e-02  6.47614141e-06
 -1.41261037e-15]
Ash excitation after gs state: [[0, 1, 2, 3], [0, 2], [1, 3], [1, 3], [0, 1, 2, 3]]
Energies: [-55.42903319816899, -55.42905574988393, -55.42907027670012, -55.429070276720495, -55.4290702767202]
Old gradient: [0.07869106190202811, 0.004949661322760376, 0.003883036832467335, 4.1696614552690775e-06, 6.642604723841594e-07]


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


In [ ]:
-55.273035076158735